# Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [31]:
import pandas as pd
data = pd.read_csv('Python_8_joins/ml-latest-small/ratings.csv')

data_userhi = data.pivot_table(index='userId',values = 'timestamp', aggfunc = 'min', fill_value = 0)
data_userbye = data.pivot_table(index='userId',values = 'timestamp', aggfunc = 'max', fill_value = 0)
data_userrates = data.pivot_table(index='userId',values = 'timestamp', aggfunc = 'count', fill_value = 0)

data_users = data_userhi.merge(data_userbye, how = 'left', on = 'userId').merge(data_userrates, how = 'left', on = 'userId')
data_users = data_users[data_users.timestamp >= 100]
data_users['LT'] = data_users.timestamp_y - data_users.timestamp_x
data_users['LT'].mean()

39331748.56653992

# Задание 2 
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
1) таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2) аналогичную таблицу по типам выручки с указанием адреса клиента. Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [42]:
# rzd - железнодорожные перевозки
# auto - автомобильные перевозки
# air - воздушные перевозки
# client_base - адреса клиентов
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
# rzd
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
# auto
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
# air
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
# client_base

In [73]:
data1 = rzd.merge(auto, how='outer').merge(air, how='outer')
data2 = data1.merge(client_base, how = 'left')

# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей (geo_logs). Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок(visits)?

0. Предположение: приложение для полноценной работы отправляет на сервер координаты через одинаковые промежутки времени. (если это не так, логика та же, однако в расчетах всех вероятностей у логов появятся веса, равные времени нахождения в локации)
1. Разделить координатную сетку, создав "ячейки" прямоугольной формы.
2. Присвоить каждому логу в geo_logs id "ячейки" по левому верхнему углу прямоугольника.
3. Сгруппировать geo_logs по ячейкам и пользователям
4. Нормированная метрика пользователя (user_mobility): Проанализировать распределение для каждого пользователя, посчитав коэффициент мобильности. Например, (количество уникальных ячеек пользователя / общее количество записей в логах по пользователю).  Используем для кластеризации пользователей: "Дом-работа"; "Домашний"; "разъездной характер".
5. Нормированная метрика пользователя-ячейки (place_atrractor): Рассчитать для каждой ячейки коэффициент частоты пребывания. Например, (количество логов для связки пользователь - id ячейки / общее количество записей в логах по пользователю)
6. В таблицу visits добавляем (left join) user_mobility, id топ-3 ячеек (в разные поля). Топ-N можно корректировать по результатам анализа распределения в п.4